In [15]:
import os


In [16]:
%pwd


'c:\\ML\\MLOPs Production Ready Deep Learning Project\\research'

In [17]:
os.chdir("../")

In [18]:
%pwd

'c:\\ML\\MLOPs Production Ready Deep Learning Project'

In [29]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [30]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories



In [31]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [32]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [33]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [34]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-11-01 20:51:17,857: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-01 20:51:17,862: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-01 20:51:17,864: INFO: common: Created directory at: artifacts]
[2025-11-01 20:51:17,867: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-11-01 20:51:17,868: INFO: 1983665874: Downloading data from https://drive.google.com/file/d/1OxBDwv_zVBXA-SQSFHs1GlC_wc-icomL/view?usp=drive_link into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1OxBDwv_zVBXA-SQSFHs1GlC_wc-icomL
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1OxBDwv_zVBXA-SQSFHs1GlC_wc-icomL&confirm=t&uuid=42e0b00b-a32a-4ef0-8b20-3c5e8ced11cd
To: c:\ML\MLOPs Production Ready Deep Learning Project\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:01<00:00, 26.9MB/s]

[2025-11-01 20:51:21,366: INFO: 1983665874: Downloaded data from https://drive.google.com/file/d/1OxBDwv_zVBXA-SQSFHs1GlC_wc-icomL/view?usp=drive_link into file artifacts/data_ingestion/data.zip]
